In [1]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [4]:
download_file_from_google_drive('181Ua_0HhqaWlyCeHD6_hFsPZhGbFKbU8', '/content/prepared.zip')

In [ ]:
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import os

In [ ]:
logging.warnings.filterwarnings('ignore')

### Genre Selection:

In [ ]:
genres = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}

### Hyper Parameters:

In [ ]:
hparams = {
    'samplerate': 22050,
    'seq_length': 256,
    'hop_length': 512,
    'fft_window': 2048,
    'num_classes': len(genres)
}

### Extract Features:

##### Generate Train Dataset if not exist:

In [ ]:
train_x = np.load('./prepared/train_x.npy', allow_pickle=False)
val_x = np.load('./prepared/val_x.npy', allow_pickle=False)
test_x = np.load('./prepared/test_x.npy', allow_pickle=False)

### Generate Ground Truth Labels:

In [ ]:
train_y = np.zeros(shape=(5 * 70 * hparams['num_classes'], hparams['num_classes']), dtype=float)
val_y   = np.zeros(shape=(5 * 20 * hparams['num_classes'], hparams['num_classes']), dtype=float)
test_y  = np.zeros(shape=(5 * 10 * hparams['num_classes'], hparams['num_classes']), dtype=float)

train_counter = 0
val_counter   = 0
test_counter  = 0

for i in range(5 * 70 * hparams['num_classes']):
    train_y[train_counter, :] = tf.keras.utils.to_categorical(train_counter // (5*70), num_classes=hparams['num_classes'])
    train_counter += 1

for i in range(5 * 20 * hparams['num_classes']):
    val_y[val_counter, :] = tf.keras.utils.to_categorical(val_counter // (5*20), num_classes=hparams['num_classes'])
    val_counter += 1
        
for i in range(5 * 10 * hparams['num_classes']):
    test_y[test_counter, :] = tf.keras.utils.to_categorical(test_counter // (5*10), num_classes=hparams['num_classes'])
    test_counter += 1

In [ ]:
train_x = np.moveaxis(train_x, 1, 2)
val_x = np.moveaxis(val_x, 1, 2)
test_x = np.moveaxis(test_x, 1, 2)

In [ ]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(64, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(64, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(64, activation='relu')
        self.L4 = tf.keras.layers.Dropout(0.3)
        self.L5 = tf.keras.layers.Dense(units=hparams['num_classes'], activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        self.x = self.L4(self.x)
        self.x = self.L5(self.x)
        return self.x

In [ ]:
model = GenreClassifierModel()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(train_x, train_y, epochs=30)

Epoch 1/30
110/110 [==============================] - 3s 31ms/step - loss: 2.2925 - accuracy: 0.1217
Epoch 2/30
110/110 [==============================] - 3s 31ms/step - loss: 2.1497 - accuracy: 0.2200
Epoch 3/30
110/110 [==============================] - 3s 30ms/step - loss: 1.9265 - accuracy: 0.3117
Epoch 4/30
110/110 [==============================] - 3s 30ms/step - loss: 1.7375 - accuracy: 0.3803
Epoch 5/30
110/110 [==============================] - 3s 31ms/step - loss: 1.5859 - accuracy: 0.4234
Epoch 6/30
110/110 [==============================] - 3s 32ms/step - loss: 1.4781 - accuracy: 0.4631
Epoch 7/30
110/110 [==============================] - 3s 31ms/step - loss: 1.3786 - accuracy: 0.4989
Epoch 8/30
110/110 [==============================] - 3s 29ms/step - loss: 1.2839 - accuracy: 0.5331
Epoch 9/30
110/110 [==============================] - 3s 30ms/step - loss: 1.1943 - accuracy: 0.5729
Epoch 10/30
110/110 [==============================] - 4s 32ms/step - loss: 1.1171 - accura

In [ ]:
model.evaluate(val_x, val_y)

32/32 [==============================] - 1s 17ms/step - loss: 3.9246 - accuracy: 0.3670


[3.924561023712158, 0.367000013589859]

In [ ]:
gno = 0

g = list(genres.keys())[gno]
print('Actual Genre', g, gno)

for i in range(hparams['num_classes']):

    m = hparams['seq_length']
    
    example = extract_features('./dataset/{}/{}.000'.format(g, g) + str(90+i) + '.wav', hparams)
    
    segment_1 = np.moveaxis(example, 0, 1)[0*m:1*m, :].reshape(1, m, 40)
    segment_2 = np.moveaxis(example, 0, 1)[1*m:2*m, :].reshape(1, m, 40)
    segment_3 = np.moveaxis(example, 0, 1)[2*m:3*m, :].reshape(1, m, 40)
    segment_4 = np.moveaxis(example, 0, 1)[3*m:4*m, :].reshape(1, m, 40)
    segment_5 = np.moveaxis(example, 0, 1)[4*m:5*m, :].reshape(1, m, 40)
    
    print('===============================================')
    print('Segment-0:', np.argmax(model.predict(segment_1)))
    print('Segment-1:', np.argmax(model.predict(segment_2)))
    print('Segment-2:', np.argmax(model.predict(segment_3)))
    print('Segment-3:', np.argmax(model.predict(segment_4)))
    print('Segment-4:', np.argmax(model.predict(segment_5)))

Actual Genre blues 0
Segment-0: 0
Segment-1: 0
Segment-2: 9
Segment-3: 9
Segment-4: 0
Segment-0: 8
Segment-1: 2
Segment-2: 0
Segment-3: 8
Segment-4: 2
Segment-0: 0
Segment-1: 0
Segment-2: 5
Segment-3: 0
Segment-4: 0
Segment-0: 5
Segment-1: 1
Segment-2: 0
Segment-3: 5
Segment-4: 0
Segment-0: 9
Segment-1: 0
Segment-2: 8
Segment-3: 4
Segment-4: 0
Segment-0: 5
Segment-1: 3
Segment-2: 0
Segment-3: 2
Segment-4: 2
Segment-0: 1
Segment-1: 1
Segment-2: 2
Segment-3: 5
Segment-4: 2
Segment-0: 8
Segment-1: 8
Segment-2: 8
Segment-3: 9
Segment-4: 9
Segment-0: 6
Segment-1: 0
Segment-2: 9
Segment-3: 0
Segment-4: 9
Segment-0: 9
Segment-1: 0
Segment-2: 8
Segment-3: 2
Segment-4: 9
